In [11]:
import sys
sys.path.append('../../src/meta_rule/')

from lnn_operators import and_lukasiewicz, or_lukasiewicz, negation
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn

In [12]:
#to train a xor we need its truth table
x = torch.from_numpy(np.array([[0, 0], \
                               [0, 1], \
                               [1, 0], \
                               [1, 1]])).float()

#the target values for each row in the truth table (xor)
y = torch.from_numpy(np.array([[0], \
                               [1], \
                               [1], \
                               [0]])).float()

In [23]:
class xorLNN(nn.Module):
    def __init__(self, alpha, arity, slack):
        super(xorLNN, self).__init__()
        self.op_and1 = and_lukasiewicz(alpha, arity, slack)
        self.op_and2 = and_lukasiewicz(alpha, arity, slack)
        self.op_or = or_lukasiewicz(alpha, arity, slack)
    
    def forward(self, x):
        x0 = x[:,0].view(-1,1)
        x1 = x[:,1].view(-1,1)
        yhat = self.op_or(torch.cat((self.op_and1(torch.cat((x0, negation(x1)), 1)), \
                            self.op_and2(torch.cat((negation(x0), x1), 1))), 1))
        return yhat

In [24]:
model = xorLNN(0.8, 2, False)

In [25]:
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

In [26]:
for iter in range(100):
    model.train()
    optimizer.zero_grad()

    yhat = model(x)
    loss = loss_fn(yhat, y)

    print("Iteration " + str(iter) + ": " + str(loss.item()))
    loss.backward()
    optimizer.step()


Iteration 0: 0.00041395798325538635
Iteration 1: 0.0003413597878534347
Iteration 2: 0.00027990160742774606
Iteration 3: 0.00022847841319162399
Iteration 4: 0.00018598556926008314
Iteration 5: 0.00015122962940949947
Iteration 6: 0.0001230025663971901
Iteration 7: 0.00010024568473454565
Iteration 8: 8.197502756956965e-05
Iteration 9: 6.734087219228968e-05
Iteration 10: 5.5627755500609055e-05
Iteration 11: 4.625439760275185e-05
Iteration 12: 3.872895103995688e-05
Iteration 13: 3.267884312663227e-05
Iteration 14: 2.7761290766648017e-05
Iteration 15: 2.3767666789353825e-05
Iteration 16: 2.051913361356128e-05
Iteration 17: 1.7821967048803344e-05
Iteration 18: 1.5646355677745305e-05
Iteration 19: 1.3783679605694488e-05
Iteration 20: 1.2263739336049184e-05
Iteration 21: 1.0967321941279806e-05
Iteration 22: 9.89442560239695e-06
Iteration 23: 8.970543603936676e-06
Iteration 24: 8.165872714016587e-06
Iteration 25: 7.480413842131384e-06
Iteration 26: 6.899264008097816e-06
Iteration 27: 6.407521595

In [28]:
#this is a hyperparameter
alpha = 0.8

op_and1 = and_lukasiewicz(alpha, 2, False)
op_and2 = and_lukasiewicz(alpha, 2, False)
op_or = or_lukasiewicz(alpha, 2, False)

#to train a xor we need its truth table
x = torch.from_numpy(np.array([[0, 0], \
                               [0, 1], \
                               [1, 0], \
                               [1, 1]])).float()

#the target values for each row in the truth table (xor)
y = torch.from_numpy(np.array([[0], \
                               [1], \
                               [1], \
                               [0]])).float()

loss_fn = nn.BCELoss()
optimizer = optim.Adam([{'params': op_or.parameters()}, \
                        {'params': op_and1.parameters()}, \
                        {'params': op_and2.parameters()}], lr=0.1)

for iter in range(100):
    op_or.train()
    op_and1.train()
    op_and2.train()
    optimizer.zero_grad()

    x0 = x[:,0].view(-1,1)
    x1 = x[:,1].view(-1,1)
    yhat = op_or(torch.cat((op_and1(torch.cat((x0, negation(x1)), 1)), \
                            op_and2(torch.cat((negation(x0), x1), 1))), 1))
    loss = loss_fn(yhat, y)

    print("Iteration " + str(iter) + ": " + str(loss.item()))
    loss.backward()
    optimizer.step()

#check to see output of xor post-training
x0 = x[:,0].view(-1,1)
x1 = x[:,1].view(-1,1)
yhat = op_or(torch.cat((op_and1(torch.cat((x0, negation(x1)), 1)), \
                        op_and2(torch.cat((negation(x0), x1), 1))), 1))
check_values = torch.cat((yhat, y), 1)
print("------- Checking outputs (left) vs ground truth (right): -----")
print(check_values.detach())

#LNN parameters: post-training (we have 3 sets of beta, argument weights)
print("--------------- LNN Parameters (post-training) ---------------")
beta_or, argument_wts_or = op_or.AND.cdd()
beta_and1, argument_wts_and1 = op_and1.cdd()
beta_and2, argument_wts_and2 = op_and2.cdd()

np.set_printoptions(precision=3, suppress=True)
print("OR (beta, argument weights): " \
      + str(np.around(beta_or.item(), decimals=3)) + " " \
      + str(argument_wts_or.detach().numpy()))
print("AND1 (beta, argument weights): " \
      + str(np.around(beta_and1.item(), decimals=3)) + " " \
      + str(argument_wts_and1.detach().numpy()))
print("AND2 (beta, argument weights): " \
      + str(np.around(beta_and2.item(), decimals=3)) + " " \
      + str(argument_wts_and2.detach().numpy()))

Iteration 0: 0.0005070384358987212
Iteration 1: 0.0004226093296892941
Iteration 2: 0.00035011349245905876
Iteration 3: 0.0002885941066779196
Iteration 4: 0.0002370504371356219
Iteration 5: 0.00019430331303738058
Iteration 6: 0.00015917410200927407
Iteration 7: 0.00013051435234956443
Iteration 8: 0.0001072355080395937
Iteration 9: 8.839827933115885e-05
Iteration 10: 7.324236503336579e-05
Iteration 11: 6.0992642829660326e-05
Iteration 12: 5.106781463837251e-05
Iteration 13: 4.3065447243861854e-05
Iteration 14: 3.6598037695512176e-05
Iteration 15: 3.130791810690425e-05
Iteration 16: 2.6941725081996992e-05
Iteration 17: 2.3410046196659096e-05
Iteration 18: 2.048934402409941e-05
Iteration 19: 1.8030596038443036e-05
Iteration 20: 1.6003998098312877e-05
Iteration 21: 1.4275432477006689e-05
Iteration 22: 1.2785292710759677e-05
Iteration 23: 1.1593181625357829e-05
Iteration 24: 1.0550087608862668e-05
Iteration 25: 9.641104952606838e-06
Iteration 26: 8.851335223880596e-06
Iteration 27: 8.1807766